# REGRESSION PARAMETERS    AND OPTIMIZATION

In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
backend module://ipykernel.pylab.backend_inline version unknown


# Load data

In [ ]:
import ShaKer.rna_tools.rna_io as rio
data = rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

for k,v in data.items():print k,"\t",  len(v[1])

# VISUALIZING DIFFERENCE IN PARAMETERS ... CALCULATE

In [ ]:
import ShaKer.simushape as sim
import xgboost
from sklearn.ensemble import RandomForestRegressor


models  = [sim.make_model(data,['23sRNA','R009']), # xgbregressor with optimized paramters  :) 
           sim.make_model(data,['23sRNA','R009'],xgboost.XGBRegressor()),
           sim.make_model(data,['23sRNA','R009'],RandomForestRegressor()),
           sim.make_model(data,['23sRNA','R009'],RandomForestRegressor(**{'oob_score': False,
                                             'min_impurity_split': 0.01,
                                             'bootstrap': True,
                                             'min_samples_leaf': 1,
                                             'n_estimators': 16,
                                             'min_samples_split': 6,
                                             'min_weight_fraction_leaf': 0.02,
                                             'max_features': None}))]


data.pop("23sRNA")
data.pop("R009")
predictions = { k: [v[0]] + [ sim.predict(model,v[1]) for model in models] for k,v in data.items() }



# ...DRAW 

In [ ]:
import notebook_helper as hlp 

for k,v in data.items()[:4]: 
    print k
    hlp.draw_seq_rea(v[1], predictions[k], v[2])

# OPTIMIZATION

In [ ]:
import ShaKer.optimization.optimize_gradient_boosting as opt
opt.optimize_xgb(opt.quickladdata(), n_jobs=2, n_iter=10) # see code

In [ ]:
import ShaKer.optimization.optimize_forest as optifo
optifo.opti_forest(optifo.quickloaddata(),  n_jobs=2, n_iter=10)